In [1]:
import numpy as np

In [2]:
def Tij(theta,i,DH):
    theta_local = theta + DH[i,3]
    Tij1 = np.array([[np.cos(theta_local), -np.sin(theta_local), 0, 0],
                     [np.sin(theta_local), np.cos(theta_local), 0, 0],
                     [0, 0, 1, DH[i,2]],
                     [0, 0, 0, 1]],dtype=np.float64)
    Tij2 = np.array([[1, 0, 0, DH[i,1]],
                     [0, np.cos(DH[i,3]), -np.sin(DH[i,3]), 0],
                     [0, np.sin(DH[i,3]), np.cos(DH[i,3]), 0],
                     [0, 0, 0, 1]],dtype=np.float64)
    return Tij1@Tij2

In [3]:
def FkineDH(qs, DH):
    T = np.identity(4)
    n = DH.shape[0]
    for i in range(n):
        T = T @ Tij(qs[i],i,DH)
    return T

In [4]:
#      0     1 2 3
#      theta a d alpha
DH = [[0.0,175.0,495.0,-np.pi/2],
      [-np.pi/2,900.0,0.0,0.0],
      [0.0,170.0,0.0,-np.pi/2],
      [0.0,0.0,960.0,np.pi/2],
      [0.0,0.0,0.0,-np.pi/2],
      [np.pi,0.0,135.0,0.0]]
DH = np.array(DH)
DH

array([[  0.        , 175.        , 495.        ,  -1.57079633],
       [ -1.57079633, 900.        ,   0.        ,   0.        ],
       [  0.        , 170.        ,   0.        ,  -1.57079633],
       [  0.        ,   0.        , 960.        ,   1.57079633],
       [  0.        ,   0.        ,   0.        ,  -1.57079633],
       [  3.14159265,   0.        , 135.        ,   0.        ]])

In [5]:
import timeit

# 正向运动学速度

In [6]:
timeit.timeit("FkineDH(np.zeros((6)),DH)", setup="import numpy as np; from __main__ import FkineDH, DH", number=10000)/10000

5.016464000000269e-05

In [7]:
def FkineDH_all(theta,DH):
    DOF = len(theta)
    T = np.identity(4)
    Ts = []
    for i in range(DOF):
        T = T @ Tij(theta[i],i,DH)
        Ts.append(T)
    return Ts

In [8]:
def T2rt(T):
    return T[0:3,0:3], T[0:3,3]

In [9]:
Te = FkineDH(np.zeros(6),DH)
Te

array([[-1.8369702e-16, -6.1232340e-17, -1.0000000e+00, -1.3500000e+02],
       [ 1.0000000e+00,  1.2246468e-16, -1.8369702e-16, -2.0350000e+03],
       [ 1.2246468e-16, -1.0000000e+00,  6.1232340e-17,  6.6500000e+02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

In [10]:
(R,p) = T2rt(Te)
R

array([[-1.8369702e-16, -6.1232340e-17, -1.0000000e+00],
       [ 1.0000000e+00,  1.2246468e-16, -1.8369702e-16],
       [ 1.2246468e-16, -1.0000000e+00,  6.1232340e-17]])

In [11]:
def wraptopi(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def wrapto1(x):
    return (x + 1) % 2 - 1

In [12]:
def se3ToVec(se3):
    return np.array([se3[0,3], se3[1,3], se3[2,3], se3[2,1], se3[0,2], se3[1,0]])

In [13]:
def VecToso3(omg):
    """Converts a 3-vector to an so(3) representation

    :param omg: A 3-vector
    :return: The skew symmetric representation of omg

    Example Input:
        omg = np.array([1, 2, 3])
    Output:
        np.array([[ 0, -3,  2],
                  [ 3,  0, -1],
                  [-2,  1,  0]])
    """
    return np.array([[0,      -omg[2],  omg[1]],
                     [omg[2],       0, -omg[0]],
                     [-omg[1], omg[0],       0]])

In [14]:
def NearZero(z):
    """Determines whether a scalar is small enough to be treated as zero

    :param z: A scalar input to check
    :return: True if z is close to zero, false otherwise

    Example Input:
        z = -1e-7
    Output:
        True
    """
    return abs(z) < 1e-6

In [15]:
def MatrixLog3(R):
    """Computes the matrix logarithm of a rotation matrix

    :param R: A 3x3 rotation matrix
    :return: The matrix logarithm of R

    Example Input:
        R = np.array([[0, 0, 1],
                      [1, 0, 0],
                      [0, 1, 0]])
    Output:
        np.array([[          0, -1.20919958,  1.20919958],
                  [ 1.20919958,           0, -1.20919958],
                  [-1.20919958,  1.20919958,           0]])
    """
    acosinput = (np.trace(R) - 1) / 2.0
    if acosinput >= 1:
        return np.zeros((3, 3))
    elif acosinput <= -1:
        if not NearZero(1 + R[2][2]):
            omg = (1.0 / np.sqrt(2 * (1 + R[2][2]))) \
                  * np.array([R[0][2], R[1][2], 1 + R[2][2]])
        elif not NearZero(1 + R[1][1]):
            omg = (1.0 / np.sqrt(2 * (1 + R[1][1]))) \
                  * np.array([R[0][1], 1 + R[1][1], R[2][1]])
        else:
            omg = (1.0 / np.sqrt(2 * (1 + R[0][0]))) \
                  * np.array([1 + R[0][0], R[1][0], R[2][0]])
        return VecToso3(np.pi * omg)
    else:
        theta = np.arccos(acosinput)
        return theta / 2.0 / np.sin(theta) * (R - np.array(R).T)

In [16]:
def RpToTrans(R, p):
    """Converts a rotation matrix and a position vector into homogeneous
    transformation matrix

    :param R: A 3x3 rotation matrix
    :param p: A 3-vector
    :return: A homogeneous transformation matrix corresponding to the inputs

    Example Input:
        R = np.array([[1, 0,  0],
                      [0, 0, -1],
                      [0, 1,  0]])
        p = np.array([1, 2, 5])
    Output:
        np.array([[1, 0,  0, 1],
                  [0, 0, -1, 2],
                  [0, 1,  0, 5],
                  [0, 0,  0, 1]])
    """
    return np.r_[np.c_[R, p], [[0, 0, 0, 1]]]

def TransToRp(T):
    """Converts a homogeneous transformation matrix into a rotation matrix
    and position vector

    :param T: A homogeneous transformation matrix
    :return R: The corresponding rotation matrix,
    :return p: The corresponding position vector.

    Example Input:
        T = np.array([[1, 0,  0, 0],
                      [0, 0, -1, 0],
                      [0, 1,  0, 3],
                      [0, 0,  0, 1]])
    Output:
        (np.array([[1, 0,  0],
                   [0, 0, -1],
                   [0, 1,  0]]),
         np.array([0, 0, 3]))
    """
    T = np.array(T)
    return T[0: 3, 0: 3], T[0: 3, 3]

def TransInv(T):
    """Inverts a homogeneous transformation matrix

    :param T: A homogeneous transformation matrix
    :return: The inverse of T
    Uses the structure of transformation matrices to avoid taking a matrix
    inverse, for efficiency.

    Example input:
        T = np.array([[1, 0,  0, 0],
                      [0, 0, -1, 0],
                      [0, 1,  0, 3],
                      [0, 0,  0, 1]])
    Output:
        np.array([[1,  0, 0,  0],
                  [0,  0, 1, -3],
                  [0, -1, 0,  0],
                  [0,  0, 0,  1]])
    """
    R, p = TransToRp(T)
    Rt = np.array(R).T
    return np.r_[np.c_[Rt, -np.dot(Rt, p)], [[0, 0, 0, 1]]]

In [17]:
def MatrixLog6(T):
    """Computes the matrix logarithm of a homogeneous transformation matrix

    :param R: A matrix in SE3
    :return: The matrix logarithm of R

    Example Input:
        T = np.array([[1, 0,  0, 0],
                      [0, 0, -1, 0],
                      [0, 1,  0, 3],
                      [0, 0,  0, 1]])
    Output:
        np.array([[0,          0,           0,           0]
                  [0,          0, -1.57079633,  2.35619449]
                  [0, 1.57079633,           0,  2.35619449]
                  [0,          0,           0,           0]])
    """
    R, p = TransToRp(T)
    omgmat = MatrixLog3(R)
    if np.array_equal(omgmat, np.zeros((3, 3))):
        return np.r_[np.c_[np.zeros((3, 3)),
                           [T[0][3], T[1][3], T[2][3]]],
                     [[0, 0, 0, 0]]]
    else:
        theta = np.arccos((np.trace(R) - 1) / 2.0)
        return np.r_[np.c_[omgmat,
                           np.dot(np.eye(3) - omgmat / 2.0 \
                           + (1.0 / theta - 1.0 / np.tan(theta / 2.0) / 2) \
                              * np.dot(omgmat,omgmat) / theta,[T[0][3],
                                                               T[1][3],
                                                               T[2][3]])],
                     [[0, 0, 0, 0]]]

In [18]:
MatrixLog6(Te)

array([[ 0.00000000e+00, -1.20919958e+00, -1.20919958e+00,
        -1.28357588e+03],
       [ 1.20919958e+00,  0.00000000e+00,  1.20919958e+00,
        -1.92445166e+03],
       [ 1.20919958e+00, -1.20919958e+00,  0.00000000e+00,
        -3.73027538e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [19]:
def my_jacob0(q,DH):
    DOF = len(q)
    J0 = []
    Ts = FkineDH_all(q,DH)
    pe = Ts[-1][0:3,3]
    for i in range(DOF):
        zi = Ts[i][0:3,2]
        pi = Ts[i][0:3,3]
        J0.append(np.concatenate((np.cross(zi,pe-pi),zi),axis=0))
    # print(J0)
    J0 = np.vstack(J0).transpose()
    return J0

In [20]:
def my_jacobe(q,DH):
    DOF = len(q)
    J0 = []
    Ts = FkineDH_all(q,DH)
    Rb = Ts[-1][0:3,0:3]
    pe = Ts[-1][0:3,3]
    for i in range(DOF):
        zi = Ts[i][0:3,2]
        pi = Ts[i][0:3,3]
        J0.append(np.concatenate((np.cross(zi,pi),zi),axis=0))
    J0 = np.vstack(J0).transpose()
    JB = np.vstack([np.hstack((Rb.transpose(),np.zeros((3,3)))),np.hstack((np.zeros((3,3)),Rb.transpose()))]) @ J0
    return JB


In [26]:
def my_ikine_Newton(Te,DH,q0):
    NMax = 10
    q = q0
    T = FkineDH(q0,DH)
    for i in range(NMax):
        error = MatrixLog6(TransInv(Te)*T)
        e = se3ToVec(error)
        JB = my_jacobe(q,DH)
        dq = np.linalg.solve(JB,e)
        # print(np.linalg.norm(e))
        q = q + dq
        T = FkineDH(q,DH)
    return q

In [27]:
my_ikine_Newton(Te,DH,np.zeros(6))

array([-9.24418051e+18,  4.80123506e+18, -7.78443818e+04, -8.63077677e+04,
       -7.83261250e+34,  7.83261250e+34])

# 逆向运动学速度

统一迭代10次，不会提前终止

In [28]:
timeit.timeit('my_ikine_Newton(Te,DH,np.zeros(6))',setup="from __main__ import DH, my_ikine_Newton, Te; import numpy as np", number=1000)/1000

0.003000376499999902